In [ ]:
%load_ext autotime
%load_ext autoreload
%autoreload 2

In [ ]:
# Used to make our results easier to read
from pprint import pprint

import pandas as pd
import requests
from apache_beam.options.pipeline_options import PipelineOptions
from IPython.display import SVG, Image, display
from test_policy.policy import demo_policy as policy_definition
from vulkan.beam.local.convert import build_beam_policy
from vulkan.core.policy import Policy

from vulkan_public.spec.dependency import INPUT_NODE

In [ ]:
import os
import platform
import signal
import socket
import subprocess


def is_server_running(host: str, port: int, timeout: float = 1.0) -> bool:
    """Check if a server is running on the given host and port.

    Args:
        host (str): The hostname or IP address of the server.
        port (int): The port number to check.
        timeout (float): The timeout for the connection in seconds (default is 1.0).

    Returns:
        bool: True if a server is running, False otherwise.
    """
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.settimeout(timeout)  # Set timeout for the connection attempt
        try:
            sock.connect((host, port))
            return True
        except (socket.timeout, ConnectionRefusedError, OSError):
            return False


def render_pipeline(
    pipeline_builder,
    output_image_path: str,
    render_port: int = -1,
):
    if render_port > 0:
        busy = is_server_running("localhost", render_port, timeout=1)
        if busy:
            msg = (
                f"There is already a server running on port {render_port}. \n"
                "Skipping starting a server for this render to avoid conflict."
            )
            print(msg)
            render_port = -1

    args = [
        "--runner=apache_beam.runners.render.RenderRunner",
        f"--render_output={output_image_path}",
        f"--render_port={render_port}",
    ]
    options = PipelineOptions(args)

    p = pipeline_builder.build(backfill_id="test-id", pipeline_options=options)
    p.run()
    return SVG(output_image_path)

In [ ]:
def run_local(pipeline_builder, run_id: str = "test-id"):
    args = [
        "--runner=DirectRunner",
    ]
    options = PipelineOptions(args)

    p = pipeline_builder.build(backfill_id=run_id, pipeline_options=options)
    run = p.run()

    output_data = pd.read_parquet(pipeline_builder.output_path)

    return output_data

In [ ]:
df = pd.read_csv("simple_bkt_lg.csv").iloc[0:1]
df["month"] = df["month"].astype(str)
df["tax_id"] = df["tax_id"].astype(str)

df.to_parquet("input.parquet")

In [ ]:
policy = Policy.from_definition(policy_definition)

In [ ]:
from vulkan_public.schemas import DataSourceSpec

test_schema = {
    "name": "vendor",
    "keys": ["tax_id"],
    "request": {
        "url": "https://jsonplaceholder.typicode.com/todos/1",
    },
    "caching": {
        "enabled": False,
    },
}

data_sources = {
    INPUT_NODE: "input.parquet",
    "vendor-name:api-name:v0.0.1": DataSourceSpec.model_validate(test_schema),
}
config_variables = {"SCORE_CUTOFF": 500}

builder = build_beam_policy(
    policy,
    data_sources=data_sources,
    output_path="./output/",
    config_variables=config_variables,
)

In [ ]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

result = run_local(builder)

result.head()